In [5]:
import os
import pandas as pd
import datetime as dt
from markupsafe import Markup
import fix_yahoo_finance as yf
yf.pdr_override()

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree, metrics, neighbors
from sklearn.model_selection import cross_val_score

from util import create_df_benchmark, fetchOnlineData, get_data, \
    symbol_to_path
from indicators import get_indicators

In [2]:
symbol = "AABA"


# Set Forecast date
forecast_date = dt.date.today()


# Get Forecast time
forecast_time = 7

# Get Lookback
forecast_lookback = 365


# Set lookback date of data to train and test
start_d = forecast_date - dt.timedelta(forecast_lookback)
start_d = f"{start_d:%Y-%m-%d}"
yesterday = dt.date.today() - dt.timedelta(1)

In [3]:
portf_value = fetchOnlineData(start_d, symbol, yesterday)

[                       0%                       ]

[*********************100%***********************]  1 of 1 downloaded

In [9]:
# Normalize the prices Dataframe
normed = portf_value.copy()
#normed = scaling_data(normed, symbol)

normed['date'] = portf_value.index
normed.set_index('date', inplace=True)
normed.rename(columns={'Adj Close': symbol}, inplace=True)


# Get indicators
sym_mom, sma, q, rsi_value = get_indicators(normed, symbol)


# Create momentum column
normed['Momentum'] = sym_mom

# Create SMA column
normed['SMA'] = sma

# Create SMA column
normed['RSI'] = rsi_value

# Clean nan values
normed = normed.fillna(0)

# Sort dataframe by index
normed.sort_index()


# Define X and y
feature_cols = ['Momentum', 'RSI']
X = normed[feature_cols]
y = normed[symbol]

# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=False)



date
2018-02-15   -0.021181
2018-02-16   -0.011962
2018-02-20    0.028686
2018-02-21    0.021488
2018-02-22    0.046563
2018-02-23    0.124512
2018-02-26    0.107832
2018-02-27    0.070745
2018-02-28    0.049642
2018-03-01    0.001086
2018-03-02   -0.022984
2018-03-05   -0.000952
2018-03-06    0.007776
2018-03-07    0.011583
2018-03-08    0.002781
2018-03-09   -0.014660
2018-03-12   -0.006660
2018-03-13   -0.002102
2018-03-14    0.031396
2018-03-15    0.077027
2018-03-16    0.099188
2018-03-19    0.059913
2018-03-20    0.051882
2018-03-21    0.027507
2018-03-22   -0.015318
2018-03-23   -0.047246
2018-03-26   -0.022563
2018-03-27   -0.038700
2018-03-28   -0.059067
2018-03-29   -0.067741
                ...   
2018-12-18   -0.115235
2018-12-19   -0.109568
2018-12-20   -0.111964
2018-12-21   -0.116807
2018-12-24   -0.104773
2018-12-26   -0.074636
2018-12-27   -0.077205
2018-12-28   -0.070268
2018-12-31   -0.065635
2019-01-02   -0.043109
2019-01-03   -0.060724
2019-01-04    0.017678
2019-0

In [8]:
# Use only to get the best parameter for max_depth
# Loop through a few different max depths and check the performance
for d in [3, 5, 10, 15, 20]:
    # Create the tree and fit it
    decision_tree = tree.DecisionTreeRegressor(max_depth=d)
    decision_tree.fit(X_train, y_train)

    # Print out the scores on train and test
    print('max_depth=', str(d))
    print(decision_tree.score(X_train, y_train))
    print(decision_tree.score(X_test, y_test), '\n')

print('Fiiting finishied')    

max_depth= 3
0.545299145436019
-15.88438138776274 

max_depth= 5
0.6801633219117347
-16.913713184197096 

max_depth= 10
0.9784245155051181
-19.29790193310834 

max_depth= 15
0.990981119646373
-18.03016108519227 

max_depth= 20
0.990981119646373
-18.41225893389626 

Fiiting finishied
